In [8]:

import tensorflow as tf
import math
import numpy as np
import read_data
from sklearn.model_selection import train_test_split

In [9]:
def angular_err(v1, v2):
	v1xv2 = tf.reduce_sum(v1*v2,1)
	v1_len = tf.cast(tf.sqrt(tf.reduce_sum(tf.square(v1), 1)),dtype=tf.float32)
	v2_len = tf.cast(tf.sqrt(tf.reduce_sum(tf.square(v2), 1)),dtype=tf.float32)
	
	val = tf.minimum( v1xv2/((v1_len* v2_len ) + 1e-10), 0.999999)
 
	degree = tf.acos(val)
	return degree * 180/ math.pi

In [10]:
class FARE_Net:
    
    def __angular_err(self, v1, v2):
        v1xv2 = tf.reduce_sum(v1*v2,1)
        v1_len = tf.cast(tf.sqrt(tf.reduce_sum(tf.square(v1), 1)),dtype=tf.float32)
        v2_len = tf.cast(tf.sqrt(tf.reduce_sum(tf.square(v2), 1)),dtype=tf.float32)
        
        val = tf.minimum( v1xv2/((v1_len* v2_len ) + 1e-10), 0.999999)
    
        degree = tf.acos(val)
        return degree * 180/ math.pi
    
    def __init__(self, far_net_model_path, e_net_model_path):
        self.far_net = tf.keras.models.load_model(far_net_model_path)
        self.e_net = tf.keras.models.load_model(e_net_model_path)
        
    def predict(self, data) :
        gaze_directions = self.far_net(data)
        probabilities = self.e_net(data[0:2])
        
        choosed_gaze = []
        choosed = []
        
        for i, prob in enumerate(probabilities):
            if prob[0] <= prob[1]:
                choosed_gaze.append(gaze_directions[i][:3])
                choosed.append("Left")
            else :
                choosed_gaze.append(gaze_directions[i][3:])
                choosed.append("Right")
        
        return choosed_gaze, choosed
    
    def validate(self, data, labels):
        print("Validating...")
        gazes, chooseds = self.predict(data=data)
        
        new_labels = []
        
        for choosed, label in zip(chooseds, labels):
            
            if choosed == "Left" :
                new_labels.append(label[:3])
            else :
                new_labels.append(label[3:])
                
                
        angular_errors = self.__angular_err(gazes, tf.cast(np.array(new_labels, dtype=float), dtype=tf.float32))
            
        print(f"Mean Angular Error => {tf.reduce_mean(angular_errors)}")
            

In [11]:
l_eye_imgs, r_eye_imgs, face_imgs, labels = read_data.load_subject_data("data_subset/original/p07")
# l_eye_imgs, r_eye_imgs, face_imgs, labels = read_data.load_subject_data("data_subset/original/p07")

In [12]:
l_eye_train, l_eye_test, r_eye_train, r_eye_test, face_train, face_test, labels_train, labels_test = train_test_split(
    l_eye_imgs, r_eye_imgs, face_imgs, labels, test_size=0.2, random_state=42)

In [13]:
fare_net = FARE_Net(e_net_model_path="models/enhanced/e_net_model.keras",far_net_model_path="models/enhanced/far_net_model.keras" )
fare_net_ori = FARE_Net(e_net_model_path="models/original/e_net_model.keras",far_net_model_path="models/original/far_net_model.keras" )

fare_net.validate(data=[l_eye_test, r_eye_test, face_test],labels=labels)
fare_net_ori.validate(data=[l_eye_test, r_eye_test, face_test],labels=labels)

Validating...
Mean Angular Error => 11.794445991516113
Validating...
Mean Angular Error => 11.255156517028809


Validating...
Mean Angular Error => 11.843276023864746  
Validating...
Mean Angular Error => 11.301008224487305  
  
Validating...
Mean Angular Error => 11.794445991516113  
Validating...
Mean Angular Error => 11.255156517028809